In [1]:
import  time
import  numpy as np
from    config          import *
from    agent           import Agent
from    environment      import Environment
from    IPython.display import clear_output
from    matplotlib      import pyplot as plt

In [2]:
grid_size = GRID_SIZE
num_col = grid_size

possibleActions = POSSIBLE_ACTIONS

action_space_dict = {
    "U" : 0,
    "D" : 1,
    "L" : 2,
    "R" : 3,
    "S" : 4
}
n_agents          = N_AGENTS
allplayerpos      = PLAYER_POS[: n_agents]
enemy_list_pos    = ENEMY_POS[: n_agents]
batch_size        = BATCH_SIZE
replay_memory_len = REPLAY_MEMORY_LEN
type = "sticky"

In [3]:
def decode_state(state_num):
    return int(state_num/num_col), state_num%num_col

def state_encode(row,col):
    return row*num_col + col 

In [ ]:
all_agents = []
for i in range(0, N_AGENTS):
    all_agents.append(Agent(i, allplayerpos[i], type = type))


initial_states = []
for agent in all_agents:
    initial_states.append(state_encode(agent.x, agent.y))

enemy_states = []
for enemy_pos in enemy_list_pos:
    enemy_states.append(state_encode(enemy_pos[0], enemy_pos[1]))
print(initial_states, enemy_states)
env = Environment(initial_states = initial_states, enemy_states = enemy_states, type = type)
env.render()
env.show_image()

In [5]:
def run():
    total_step = 0
    rewards_list = []
    timesteps_list = []
    total_steps = 1
    for episode in range(1, 150):
        print("Episode number: ", episode)

        reward_all = 0
        time_step = 1
        for agent in all_agents:
            agent.terminal = False

        [old_map, player_states, enemy_states] = env.reset()
        print(player_states,"enemy states: ", enemy_states)
        for agent in all_agents:
            agent.set_pos(allplayerpos[agent.index])

        done = [False for _ in range(n_agents)]
        is_terminal = [False for _ in range(n_agents)]

        while not all(is_terminal):
            for agent in all_agents:
                if total_step % agent.update_rate == 0:
                    agent.update_target_model()

            print(" is_terminal",  is_terminal)

            # env.render(clear=True)
            actions = []
            for agent in all_agents:

                actions.append(agent.act(old_map, possibleActions))

            new_map, next_states, rewards, done = env.step(actions)

            for agent in all_agents:
                agent.set_pos(decode_state(next_states[agent.index]))

            for agent in all_agents:
                agent.store(new_map, rewards[agent.index], \
                done[agent.index], old_map, action_space_dict[actions[agent.index]])

                if done[agent.index] == True:
                    agent.terminal = True
                    is_terminal[agent.index] = True
                    print("agent reached landmark--------------------------------", agent.index)

            print("actions", actions)

            for agent in all_agents:
                print("agent epsilon ", agent.epsilon, "agent memory len",\
                    len(agent.expirience_replay), "steps ", time_step,\
                    "reward", rewards[agent.index], "next state ", next_states[agent.index], "agent position ",\
                    agent.return_coordinates())

            if time_step >= TIME_STEPS:
                print("------------------------------------max steps reached")
                break

            old_map = new_map

            total_step += 1
            time_step += 1
            total_steps+1
            reward_all += sum(rewards)

            if all(is_terminal):
                print("-----------------------------------all agents reached landmark--------------------------------")
                for agent in all_agents:
                    agent.save_model()
        print("total rewards", reward_all)
        for agent in all_agents:
            agent.retrain(episode)

        rewards_list.append(reward_all)
        timesteps_list.append(time_step)


In [ ]:
run()